In [4]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [9]:
%env ASTRA_DB_SECURE_BUNDLE_PATH=C:\Users\lenovo\Desktop\DB\secure-connect-advanced-db.zip‪
%env ASTRA_DB_APPLICATION_TOKEN=C:\Users\lenovo\Desktop\DB\Advanced_Db-token (1).json

env: ASTRA_DB_SECURE_BUNDLE_PATH=C:\Users\lenovo\Desktop\DB\secure-connect-advanced-db.zip‪
env: ASTRA_DB_APPLICATION_TOKEN=C:\Users\lenovo\Desktop\DB\Advanced_Db-token (1).json


In [6]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you download your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-advanced-db.zip'
}

# This token JSON file is autogenerated when you download your token, 
# if yours is different update the file name below
with open("Advanced_Db-token (1).json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('Movies')

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.0.6816


In [2]:
import os

def image_to_blob(image_path):
   
    with open(image_path, "rb") as file:
        return file.read()

def update_movie_posters(session, folder_path):
    try:
       
        poster_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]

       
        rows = session.execute("SELECT Id, name FROM Movie").all()

        if len(poster_files) < len(rows):
            print(f"Not enough posters to match with the movies.")
            return

       
        for (movie_id, movie_name), poster_file in zip(rows, poster_files):
            poster_path = os.path.join(folder_path, poster_file)

            
            poster_blob = image_to_blob(poster_path)

            
            query = "UPDATE Movie SET movie_poster = %s WHERE Id = %s"
            session.execute(query, (poster_blob, movie_id))

            print(f"Successfully updated the poster for {movie_name} with {poster_file}")

    except Exception as e:
        print(f"An error occurred while updating posters: {str(e)}")



folder_path = r"C:\\Users\\lenovo\\Desktop\\Posters"

update_movie_posters(session, folder_path)


Successfully updated the poster for Creed 3 with Creed_III_poster.jpg
Successfully updated the poster for Forrest Gump with Forest Gump.jpg
Successfully updated the poster for Inception with Inception.jpg


In [9]:
import os

def blob_to_image(blob_data, output_path):
    
    with open(output_path, "wb") as file:
        file.write(blob_data)

def query_movies_by_person(session, role, person, output_folder):
   
    try:
        print(f"Searching for movies with {role}: {person}")

        
        query = "SELECT * FROM Movie ALLOW FILTERING;"
        rows = session.execute(query)
        
        
        os.makedirs(output_folder, exist_ok=True)

        found_movies = False
        
        
        for row in rows:
            movie_id = row.id
            movie_name = row.name
            movie_cast = row.movie_cast
            movie_poster = row.movie_poster
            
            
            if role in movie_cast and person in movie_cast[role]:
                found_movies = True
                print(f"Movie ID: {movie_id}")
                print(f"Name: {movie_name}")
                print(f"Cast: {movie_cast}")

                
                if movie_poster:
                    sanitized_name = "".join(c if c.isalnum() else "_" for c in movie_name)  
                    output_path = os.path.join(output_folder, f"{sanitized_name}_poster.jpg")
                    blob_to_image(movie_poster, output_path)
                    print(f"Poster saved to {output_path}")
                else:
                    print("No poster available for this movie.")
                
            

        if not found_movies:
            print(f"No movies found for {role}: {person}.")

    except Exception as e:
        print(f"An error occurred while querying movies: {str(e)}")



output_folder = r"C:\Users\lenovo\Desktop\Test"
role = input("Enter the role to search for (director/actors): ").strip().lower()
person_name = input("Enter the name of the person: ").strip()

query_movies_by_person(session, role, person_name, output_folder)


Enter the role to search for (director/actors):  actors
Enter the name of the person:  Tom Hanks


Searching for movies with actors: Tom Hanks
Movie ID: 2
Name: Forrest Gump
Cast: {'actors': 'Tom Hanks, Robin Wright', 'director': 'Robert Zemeckis', 'music': 'Alan Silvestri'}
Poster saved to C:\Users\lenovo\Desktop\Test\Forrest_Gump_poster.jpg
